In [1]:
import requests
import csv
import json
import re
import ast

In [2]:
url = 'https://nookplaza.net/main.dart.js?v=3'
r = requests.get(url)
mainJs = r.text

C.mm=new H.b(13,{"en-us":"bunny hood","en-gb":"bunny hood","de-de":"Hasenkapuze","es-es":"gorro de conejito","fr-fr":"cagoule lapin","fr-ca":"cagoule lapin","it-it":"cappello da coniglio","nl-nl":"konijnenmuts","zh-cn":"\u5154\u5b50\u5934\u5957","zh-tw":"\u5154\u5b50\u982d\u5957","ja-jp":"\u30a6\u30b5\u30ae\u306e\u304b\u3076\u308a\u3082\u306e","ko-kr":"\ud1a0\ub07c \ubaa8\uc790","ru-ru":"\u0448\u0430\u043f\u043a\u0430 \u0437\u0430\u0439\u043a\u0438"},C.a,[P.a,P.a])

In [3]:
# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
print(len(m))

5715


In [4]:
m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]
print(len(m))

5713


In [5]:
nameMap = {}
for item in m:
    nameMap[item["en-us"].lower()] = item

In [86]:
nameMap['street organ']

{'de-de': 'Drehorgel',
 'en-gb': 'street organ',
 'en-us': 'street organ',
 'es-es': 'organillo en carro',
 'fr-ca': 'orgue de barbarie',
 'fr-fr': 'orgue de barbarie',
 'it-it': 'organo a rullo',
 'ja-jp': 'てまわしオルガン',
 'ko-kr': '배럴 오르간',
 'nl-nl': 'draaiorgel',
 'ru-ru': 'шарманка',
 'zh-cn': '手摇风琴',
 'zh-tw': '手搖風琴'}

In [7]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

image
布置大小

改造顏色
物品分類
tag
主題
可互動否

In [12]:
output = [['id', '名稱(英文)', '名稱(中文)', '買價', '賣價', '分類', '標籤', '主題', '可互動', '大小', '取得方式', 'DIY', '款式改造', '樣式改造']]
for item in data:
    if item["category"] == "Art":
        continue
    name = nameMap[item['name'].lower()]
    nameMap[item['name'].lower()]['id'] = item['content']['internalID']
    
    # 名稱 買價 賣價
    row = [item['content']['internalID'], name['en-us'], name['zh-tw'], item['content']['buy'], item['content']['sell']]

    # 分類 標籤 主題
    # 'category' 'tag' 'themes'
    if "None" in item['content']['themes']:
        item['content']['themes'].remove("None")
    row += [item['content']['category'], item['content']['tag'], '、'.join(item['content']['themes'])]
    
    # 互動 大小
    row += [item['content']['interact'], '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])]

    # 取得方式 DIY
    if not 'dIY' in item['content']:
        print(item)
    row += ['、'.join(item['content']['obtainedFrom']), item['content']['dIY']]
        
    # 改造-款式 樣式
    # bodyCustomize bodyColor bodyTitle; patternCustomize patternColor patternTitle
    row += [item['content']['bodyCustomize'], item['content']['patternCustomize']]
    
    for variation in item['variations']:
        content = variation['content']
        pass
    
#     print(row)
    output.append(row)

In [13]:
with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    csvout.writerows(output)